In [24]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
import time
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd 
import os 
import gc
import numpy as np
from PIL import Image
import cv2 as cv
from keras.models import Sequential 
from keras.layers import Conv2D
from keras.layers import MaxPooling2D 
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau
import matplotlib.pyplot as plt

# Build folder

In [25]:
#build folder
#os.makedirs('image_from_linkedin')       
#os.makedirs('image_from_linkedin/engineer')
#os.makedirs('image_from_linkedin/not_engineer')


# Define function

In [26]:
# decode url
def Replace(String):     
    return String.replace('amp;v=beta&amp;','v=beta&')
# save image to folder
def save_image(image_name, image_url, src):
    with open('image_from_linkedin/{}/'.format(src) + image_name + '.jpg', 'wb') as f:
        res = requests.get(image_url)
        f.write(res.content)

In [27]:
# capture the face
def detect_face(job, order, train_postition):
    image_path = 'image_from_linkedin/{}/'.format(job)
    image_array = cv.imread(image_path + str(order))
    #face recognization package
    face_cascade = cv.CascadeClassifier(r'C:\Users\ChadChang\AppData\Local\Programs\Python\Python36\Lib\site-packages\cv2\data\haarcascade_frontalface_default.xml')
    face = face_cascade.detectMultiScale(image_array) # it can get muti face
    if len(face) > 0 :
        x,y,w,h = face[0]
        processed_image = Image.open(image_path + str(order)).crop((x,y,x+w,y+h)).resize((64,64))
        processed_image.save('training/{}/{}'.format(train_postition, order))

In [28]:
# crawl data from Linkedin
def crawler(search_job, amount, abbreviation):
    driver = webdriver.Chrome(executable_path=r'/Users/chadchang/Desktop/chromedriver') # chrome瀏覽器 , options  = chrome_options
    driver.get('https://www.linkedin.com')
    
    # enter account
    elem_user = driver.find_element_by_name("session_key")
    elem_user.send_keys('#####')    
    
    # enter password
    elem_pass = driver.find_element_by_name("session_password")
    elem_pass.send_keys('#####')    
    driver.find_element_by_id("login-submit").click() 
    
    # click text and enter job title
    search_text = driver.find_element_by_id('nav-search-artdeco-typeahead')
    search_text.click()
    action = ActionChains(driver)
    ActionChains(driver).send_keys(search_job).perform()                         
    driver.find_element_by_class_name('search-typeahead-v2__button').click()
    
    name = []
    src = []
    total_photo = 0
    
    #ignore former 2 pages due to it has other noise informations
    for _ in range(2):
        time.sleep(2)
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(1)
        driver.find_element_by_class_name('next-text').click()
    
    #crawler page information
    while total_photo < amount:
        time.sleep(2)
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(1)
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(1)
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(1)
        soup = BeautifulSoup(driver.page_source, 'lxml')
        current = 0
        for i in soup.select('.loaded'):
            src.append(Replace(i['src']))
            name.append(i['alt'])
            total_photo +=1
        driver.find_element_by_class_name('next-text').click()
    driver.close()
    data = pd.DataFrame({'Photo_id':name, 'Url' : src})
    data = data.drop_duplicates('Photo_id').reset_index()  
    data['order'] = data['index']
    data.drop('index', axis = 1)
    temp = data.apply(lambda x : save_image(str(x['order']), x['Url'], abbreviation) , axis=1)
    del temp
    gc.collect()
    data.to_csv('Data_From_Linkedin_{}.csv'.format(abbreviation), encoding='utf-8', index = 0)


In [ ]:
crawler('工程師', 500, 'engineer')
crawler('業務', 500, 'sales')

In [47]:
data_sale = pd.read_csv('Data_From_Linkedin_sales.csv', encoding= 'utf-8')
data_engineer = pd.read_csv('Data_From_Linkedin_engineer.csv', encoding= 'utf-8')

,Photo_id,Url,order
0,Ailing Hou,https://media.licdn.com/dms/image/C5103AQGBmFL...,0
1,李侑晉 Louis,https://media.licdn.com/dms/image/C5103AQFeRfv...,1
2,段先緯,https://media.licdn.com/dms/image/C5603AQH3wTW...,2
3,Duncan Li,https://media.licdn.com/dms/image/C4E03AQH065I...,3
4,林永裕,https://media.licdn.com/dms/image/C5103AQGTOYi...,4
5,蔡忠錦,https://media.licdn.com/dms/image/C4D03AQEUnBa...,5
6,pang pang,https://media.licdn.com/dms/image/C5603AQFoBqf...,6
7,陳孟甲,https://media.licdn.com/dms/image/C5103AQH5CtA...,7
8,Robert Wu,https://media.licdn.com/dms/image/C5603AQHTZW5...,8
9,Kevin Lee,https://media.licdn.com/dms/image/C4E03AQHG2ix...,9


In [ ]:
for i in os.listdir('image_from_linkedin/engineer'):
    detect_face('engineer', i, 'engineer')

In [ ]:
for i in os.listdir('image_from_linkedin/sales'):
    detect_face('sales', i, 'not_engineer')

# Model

In [ ]:
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape = (64, 64, 3), activation = 'relu'))
#model.add(Conv2D(32, (3, 3), activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))
#model.add(Dropout(0.25))

model.add(Conv2D(32, (3, 3), activation = 'relu'))
#model.add(Conv2D(32, (3, 3), activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))
#model.add(Dropout(0.25))

model.add(Flatten())

#model.add(Dense(units = 128, activation = 'relu')) 
model.add(Dense(units = 128, activation = 'relu'))
model.add(Dense(units = 2, activation = 'softmax'))


model.compile(optimizer = 'adam', loss ='categorical_crossentropy', metrics = ['accuracy'])

In [ ]:
#Use ImageDataGenerator to produce more data and standarization
train_datagen = ImageDataGenerator(rescale = 1./255, shear_range = 0.2, zoom_range = 0.2, horizontal_flip = True)
validating_datagen = ImageDataGenerator(rescale = 1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

In [ ]:
training_set = train_datagen.flow_from_directory('training/', target_size = (64, 64), batch_size = 10, class_mode = 'categorical')
validating_set = validating_datagen.flow_from_directory('validation/', target_size = (64,64), batch_size = 10, class_mode = 'categorical')
testing_set = test_datagen.flow_from_directory('testing/', target_size = (64,64), batch_size = 30, class_mode = 'categorical')

In [ ]:
learning_rate_reduction = ReduceLROnPlateau(monitor='val_acc', patience=3, verbose=1, factor=0.5, min_lr=0.00001)

In [ ]:
history = model.fit_generator(training_set, nb_epoch=30, steps_per_epoch = 30, verbose = 1, validation_data = validating_set, validation_steps = 10)

# Plot result

In [ ]:
fig, ax = plt.subplots(1,2)
ax[0].plot(history.history['acc'], label = 'train_acc')
ax[0].plot(history.history['val_acc'], label = 'val_acc')
legend = ax[0].legend(loc = 'best')

ax[1].plot(history.history['loss'], label = 'train_loss')
ax[1].plot(history.history['val_loss'], label = 'val_loss')
legend = ax[1].legend(loc = 'best')